In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.7 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


# 1.Definimos la tarea y modelo. Seleccionamos el AutoModel

In [4]:
from transformers import AutoModelForSequenceClassification
model_name = "BSC-TeMU/roberta-base-bne"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 2. Preparamos los datos para el fine-tuning

## 2.1 Descargamos los datos

In [5]:
from datasets import load_dataset

dataset = load_dataset("mteb/amazon_reviews_multi", "es")

dataset


Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})

## 2.2 Visualizamos

In [6]:
import random
import pandas as pd
from datasets import ClassLabel
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    "Taken from https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"

    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,id,text,label,label_text
0,es_0453921,"Cómodas pero no transpiran\n\nSon cómodas pero le huele muy mal los pies con ellas, tengo que lavarlas cada 3 días,es la pega.",1,1
1,es_0335016,NO LLEGA LADRON\n\nNo ne a llegado y lo e pagado el vendedor no contesta los mensajes es ladron timador,0,0
2,es_0385381,"Decepcionada\n\nNo me gustaron. Grandes para cualquiera y mala calidad. No es como la foto porque es imposible que quede así, es extremadamente grande. Es enorme. Compré dos para regalo y quedé fatal",0,0
3,es_0751006,Para una persona\n\nGenial para tortillas de una persona No pes nada,3,3
4,es_0463103,"No es lo que esperaba\n\nCuando alguien compra Moulinex no esperaba que en la caja ponga bien grande Made in China, que hoy en día todo se fabrica allí, ya lo sé. Compré está batidora por los buenos comentarios aunque hay que destacar que son de los que reciben el aparato gratis.... Pues cuando llegó a casa, además del país donde la han hecho, encuentro una batidora con malas terminaciones y lo peor es que el final de la batidora, lo que podemos llamar la tulipa, donde van dentro las cuchillas, todo esa circunferencia corta al roce, no está pulida, no es seguro y va a rallar todo donde apoyemos la batidora. Sinceramente cuando voy en busca de más potencia en una batidora busco también calidad y aunque no sea muy cara, para tener esa terminación ya podía valer 10 euros... No la recomiendo para nada.",1,1
5,es_0233157,Cartuchos\n\nYo los tuve q devolver después de probarlos unos meses xq algunos no funcionaban en la impresora,2,2
6,es_0319995,poca voz\n\npoca voz y incomodas en las oreja sujetan mal y se caen,1,1
7,es_0193608,Mala calidad\n\nMuy malos no pintan bien y se parte la punta enseguida,0,0
8,es_0605986,"UN POCO INCÓMODO DE USAR\n\nLo mejor de este producto es el tamaño del cubo que se puede guardar en cualquier sitio, aunque pesa un poco más de lo que sería deseable, por lo que haberle instalado ruedas no hubiera estado de más. Lo peor para mi gusto es el peso de la mopa una vez mojada y aunque se escurra sigue siendo pesada. El sistema manual de bloqueo del giro del palo parece poco fiable y fácil de romper además de incómodo, puesto que está situado en lugar muy bajo del palo teniendo que agacharte para bloquearlo o bien levantar el palo para no agacharte. Es un fallo que se hubiera podido arreglar evitar muy fácilmente colocándolo en una sección superior del palo. También es incómodo porque tienes que apretar mucho la pieza de plástico con la mano para que no gire la mopa mientras friegas el suelo. Un sistema de bloqueo mucho mejor es el que tiene Leifheit en sus friegasuelos giratorios, que es un sistema de palanca mucho más fiable y cómodo. Como tengo ambas mopas puedo compararlas y sin duda le doy el premio al sistema de palanca de Leifheit. El sistema de cesta elevable de tatay para escurrir la mopa ya veremos lo que dura y cuándo se me estropee haré nueva valoración. El tapón en la base del cubo para vaciarlo me parece una tremenda tontería puesto que nadie quiere mojarse los dedos con agua sucia al quitarlo y siempre te mojas cuando lo quitas. Jamás lo utilizo y vacío el cubo de forma convencional.",2,2
9,es_0558014,"Muy variado\n\nMuchos accesorios divertidos y distintos. Tienes que pegar tú los palitos, pero pegan bastante bien. Vienen dos tipos de pegamento : cinta de doble cara para los carteles y complementos más grandes y gotas transparentes para los más pequeños. Sobra algun palo y adhesivos también. Me ha gustado mucho. Es para 50 años pero se puede aprovechar para otras edades por que hay mucha variedad.",3,3


In [7]:
dataset.set_format("pandas")
df = dataset["train"][:]
df.head()

,id,text,label,label_text
0,es_0491108,television Nevir\n\nNada bueno se me fue ka pa...,0,0
1,es_0869872,Dinero tirado a la basura con esta compra\n\nH...,0,0
2,es_0811721,solo llega una unidad cuando te obligan a comp...,0,0
3,es_0359921,PRODUCTO NO RECIBIDO.\n\nNo entro en descalifi...,0,0
4,es_0068940,Devuelto\n\nLlega tarde y co la talla equivocada,0,0


In [8]:
df["label"].value_counts()

0    40000
1    40000
2    40000
3    40000
4    40000
Name: label, dtype: int64

In [9]:
dataset.reset_format()

## 2.3 Preprocesamos los datos

In [10]:
dataset = dataset.filter(lambda x : x["label"] != 2)

Filter:   0%|          | 0/200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [11]:
def merge_star_ratings(examples):
    if examples["label"] <= 1:
        label = 0
    else:
        label = 1
    return {"labels": label}
dataset = dataset.map(merge_star_ratings)

Map:   0%|          | 0/160000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

## 2.4 Tokenizamos (preparamos para que sea input)

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [13]:
def tokenize_reviews(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

In [14]:
columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_reviews, batched=True,  remove_columns=columns)
encoded_dataset

Map:   0%|          | 0/160000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
})

# 3. Métrica de evaluación

In [15]:
from datasets import load_metric

metric = load_metric("accuracy")
metric

<ipython-input-15-87016092d90c>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

In [16]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# OJO!! EVALUACION ANTES DEL FINE TUNING

In [17]:
from transformers import Trainer

trainer_pretrained = Trainer(
    model=model,
    compute_metrics=compute_metrics
)
results = trainer_pretrained.evaluate(eval_dataset=encoded_dataset["validation"])

print("Accuracy del modelo preentrenado:", results['eval_accuracy'])

Accuracy del modelo preentrenado: 0.5


# 4. Definimos argumentos de Fine-tunning

In [18]:
from transformers import TrainingArguments

batch_size = 16
num_train_epochs = 2
num_train_samples = 2_000
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results2",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps
)

# 5. Instanciamos el trainer

In [19]:
trainer_finetune = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer
)

# 6. Entrenamos

In [20]:
trainer_finetune.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.187300,0.192497,0.936500
2,0.147300,0.227704,0.939000


TrainOutput(global_step=250, training_loss=0.2000208264067769, metrics={'train_runtime': 447.852, 'train_samples_per_second': 8.932, 'train_steps_per_second': 0.558, 'total_flos': 957938019242880.0, 'train_loss': 0.2000208264067769, 'epoch': 2.0})

In [21]:
results_2 = trainer_finetune.evaluate(eval_dataset=encoded_dataset["validation"])
print("Accuracy del modelo fine-tuned:", results_2['eval_accuracy'])

Accuracy del modelo fine-tuned: 0.939


# 7. Salvamos el modelo

In [22]:
# Save the model to a local directory
trainer_finetune.save_model("tuned/fine_tuned")

# 8. Cargamos el modelo

In [23]:
from transformers import pipeline

local_path = "tuned/fine_tuned"

clasificador = pipeline(task="text-classification", model=local_path)

# 9. Hacemos predicciones con el modelo finetuned

In [24]:
texto = "la mesa tiene las patas bonitas"

clasificador(texto)

[{'label': 'LABEL_1', 'score': 0.9062827825546265}]

In [25]:
texto2 = "me parecen bonitas las patas de la mesa"

clasificador(texto2)

[{'label': 'LABEL_1', 'score': 0.9849262237548828}]

In [26]:
texto1 = "la mesa tiene las patas mal pintadas"

clasificador(texto1)

[{'label': 'LABEL_0', 'score': 0.9851771593093872}]